- Balanced Radial Topology
- 10MW 40개, 해상 변전소 후보는 9개 
- 교차 방지를 위한 알고리즘과, Callback 함수를 통한 Lazy Constranint 추가 

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import matplotlib.pyplot as plt
import math

# 데이터 불러오기
- 10MW 터빈 총 40개 정의
- 보안 상의 이유로 데이터 및 시각화 결과는 삭제

In [ ]:
turbine_pos_10M = np.array() # 10MW 40개의 터빈의 좌표를 가지고 있는 2차원 행렬 정의 
turbine_pos_10M = np.delete(turbine_pos_10M, 0, axis=1)
turbine_pos_10M[:, 0:] /= 1000

In [ ]:
# 해상변전소 좌표 설정 (임의)
substations = []
sets = [(27, 28), (35, 36), (34, 35), (33, 34)] # 중점, 영역 안쪽 4개 substation
for (a, b) in sets:
    substations.append((turbine_pos_10M[a-1, :] + turbine_pos_10M[b-1, :])/2)

substations = np.array(substations)
new_x = (turbine_pos_10M[35, 0]*3 - turbine_pos_10M[34, 0])/2
new_y = (turbine_pos_10M[35, 1]*3 - turbine_pos_10M[34, 1])/2
substations = np.insert(substations, 1, np.array([new_x, new_y]), axis=0)
temp = substations[1:, :].copy()
temp[:, 0] += 2
temp[:, 1] -= 0.5
substations_10M = np.vstack((substations, temp))


# 육지변전소 좌표 설정 (임의)
onshore_sub = np.array([substations_10M[5, 0]+16, turbine_pos_10M[38, 1]])

# substations 시각화
plt.figure(figsize=(15, 5))
plt.scatter(turbine_pos_10M[:, 0], turbine_pos_10M[:, 1], color='C0', label='10MW')
plt.scatter(substations_10M[:, 0], substations_10M[:, 1],  color='C1', label='10MW_OFFSUB')
plt.scatter(onshore_sub[0], onshore_sub[1], color='C2', label='10MW_ONSUB')

for s in range(len(turbine_pos_10M)):
    plt.text(turbine_pos_10M[s, 0], turbine_pos_10M[s, 1], str(s+1))

for itm, s in enumerate(substations_10M):
    plt.text(s[0], s[1], f'SUB_{itm+1}')
plt.text(onshore_sub[0], onshore_sub[1], 'ON_SUB')
plt.legend( bbox_to_anchor=(1.3, 0.5))
plt.title('Turbines with 9 substations')
plt.show()

In [5]:
nodes_10M = np.vstack((substations_10M, turbine_pos_10M)) # 상위 9개는 substation
num_nodes_10M = nodes_10M.shape[0]
num_sub_10M = substations_10M.shape[0]
num_turbines = num_nodes_10M - 9 
power_production = {i:1 for i in range(9, num_nodes_10M)}

print(f'Number of 10MW turbines and substation : {num_nodes_10M}')
print(f'Number of substations : {num_sub_10M}')
print(f'Number of 10MW turbines : {num_turbines}')

substations_index = list(range(9))
turbines_index = list(range(9,49))

Number of 10MW turbines and substation : 49
Number of substations : 9
Number of 10MW turbines : 40


# 교차방지
- 10MW일때, 10개의 string으로 제약을 걸면 많은 교차 발생

In [4]:
# 교차 방지 검출을 위한 CCW 알고리즘
def ccw(p1, p2, p3) -> int:
    ret = (p1[0] * p2[1] + p2[0] * p3[1] + p3[0] * p1[1]) - (p2[0] * p1[1] + p3[0] * p2[1] + p1[0] * p3[1])
    if ret<0:
        return -1
    elif ret>0:
        return 1
    return 0

def crossing_check(a, b, c, d) -> bool:
    
    ab = ccw(a, b, c) * ccw(a, b, d)
    cd = ccw(c, d, a) * ccw(c, d, b)

    # 일직선 상 / 점을 공유하는 경우
    if ab==0 and cd==0:
        # a, b / c, d 순서로 위치 일반화 
        if a[0] > b[0] :
            a, b = b, a
        if c[0] > d[0] :
            c, d = d, c
        # 끝점 공유 (a - b,c - d)
        if np.array_equal(a, d) or np.array_equal(b, c):
            return False

        # 끝점 공유 (a,c - b - d)
        if np.array_equal(b, d) or np.array_equal(a, c):
            if np.array_equal(b, d):
                if ccw(b, a, c)==0:
                    return True
                else:
                    return False
            if np.array_equal(a, c):
                if ccw(a, b, d)==0:
                    return True
                else:
                    return False
        
        return (c[0]<=b[0]) and (a[0]<=d[0])

    if np.array_equal(a, c) or np.array_equal(a, d) or np.array_equal(b, c) or np.array_equal(b, d):
        return False
    
    return ab <=0 and cd <= 0

def get_crossing_pair(nodes, connections) -> list:
    crossing_pair = []
    for i in range(len(connections)):
        p1 = nodes[connections[i][0]]
        p2 = nodes[connections[i][1]]
        for j in range(i+1, len(connections)):
            p3 = nodes[connections[j][0]]
            p4 = nodes[connections[j][1]]
            if crossing_check(p1, p2, p3, p4):
                crossing_pair.append([connections[i], connections[j]])
    
    return crossing_pair

## 콜백 함수 구현

In [6]:
previous_incumbent = float('inf') # 초기화
num_nodes = num_nodes_10M
def no_crossing_callback(model, where):
    global previous_incumbent
    # 새로운 해를 발견했을때
    if where == GRB.Callback.MIPSOL:
        # 현재 incumbent 가져오기
        incumbent = model.cbGet(GRB.Callback.MIPSOL_OBJBST)
        # 비교, 더 나은 해 일때
        if incumbent < previous_incumbent:
            print("New incumbent found:", incumbent)
            # 갱신
            previous_incumbent = incumbent

            # 솔루션값 가져오기 (y 값만)
            solution = model.cbGetSolution(y)
            y_values = np.zeros((num_nodes, num_nodes))
            for (i, j), value in solution.items():
                y_values[i, j] = value

            # 현재 솔루션에서 connections 값 추출
            connections = []
            for i in range(num_nodes):
                for j in range(num_nodes):
                    if i != j and y_values[i, j] > 0.5:
                        connections.append((i, j))

            # 현재 솔루션의 connections에서 교차 체크
            nodes = nodes_10M.copy()
            crossing_pair = get_crossing_pair(nodes, connections)

            # 교차하는 간선에 대해 제약조건 추가 (y값의 합이 1 이하, 즉 최대 한개만 선택될 수 있도록 제약 추가)
            if len(crossing_pair) != 0:
                for pair in crossing_pair:
                    (i, j), (h, k) = pair
                    try:
                        model.cbLazy(y[i, j] + y[j, i] + y[h, k] + y[k, h] <= 1)
                        print(f'New constraint added: y[{i}, {j}]+y[{j}, {i}]+y[{h}, {k}]+y[{k}, {h}] <= 1' )
                    except Exception as e:
                        print(f"Error adding constraint: {e}")

# 모델

In [ ]:
cable_types = [1, 2, 3, 4, 5] # 5가지의 케이블 타입
cable_capacity_8M = {1: 3.68, 2: 4.19, 3: 6.34, 4: 8.06, 5: 9} # 각 케이블 타입의 Capacity
cable_cost = {1: 64, 2: 68, 3: 75, 4: 90, 5: 100} # Cost 다운 스케일링(1/1000)
num_strings = 10 # String 개수는 10개 (즉, 한개의 String 당 4개의 터빈 연결결)

off_on_cost = np.array([np.linalg.norm(substations_10M[i] - onshore_sub)* 180 for i in range(9)])

In [ ]:
model_10M = gp.Model('CableRoutingOptimization10M')

model_10M.setParam('TimeLimit', 3600) # Time Limit
model_10M.setParam('LazyConstraints', 1) # Lazy Constriant 추가를 위한 제약

# 결정 변수 정의 
x = model_10M.addVars(num_nodes_10M, num_nodes_10M, cable_types, vtype=GRB.BINARY, name='x')
y = model_10M.addVars(num_nodes_10M, num_nodes_10M, vtype=GRB.BINARY, name='y')
f = model_10M.addVars(num_nodes_10M, num_nodes_10M, vtype=GRB.CONTINUOUS, lb=0, name='f')
u = model_10M.addVars(num_sub_10M, vtype=GRB.BINARY, name='u')

# 목적함수
# 식(1)
model_10M.setObjective(gp.quicksum(cable_cost[t] * np.linalg.norm(nodes_10M[i]-nodes_10M[j], ord=2) * x[i, j, t] 
                                  for i in range(num_nodes_10M) 
                                  for j in range(num_nodes_10M) 
                                  for t in cable_types if i!=j)\
                      + gp.quicksum(off_on_cost[k] * u[k] for k in range(num_sub_10M)), GRB.MINIMIZE)

# 제약조건
# 식(2)
for i in range(num_nodes_10M):
    for j in range(num_nodes_10M):
        if i!=j:
            model_10M.addConstr(gp.quicksum(x[i, j, t] for t in cable_types) == y[i, j], 
                               name=f'cable_type_selection_{i}_{j}')

# 식(3)
for h in range(9, num_nodes_10M):
    model_10M.addConstr(gp.quicksum(f[h, i] - f[i, h] for i in range(num_nodes_10M) if i!=h) == power_production[h], 
                       name=f'flow_conservation_{h}')

# 식(4)
for i in range(num_nodes_10M):
     for j in range(num_nodes_10M):
          if i!=j:
               model_10M.addConstr(gp.quicksum(cable_capacity_8M[t] * x[i, j, t] for t in cable_types) >= f[i, j], 
                                  name=f'capacity_constraint_{i}_{j}')

# 식(5)
for h in range(9, num_nodes_10M):
    model_10M.addConstr(gp.quicksum(y[h, j] for j in range(num_nodes_10M) if j!=h) == 1, name=f'one_outgoing_cable_{h}')

# 식(6)
for h in range(num_sub_10M):
    model_10M.addConstr(gp.quicksum(y[h, j] for j in range(num_nodes_10M) if j!=h) == 0, name='no_outgoing_cable_substation')

# 해상변전소 제약조건
model_10M.addConstr(gp.quicksum(u[i] for i in range(num_sub_10M)) == 1, name='one_offshore_substation')

# 해상변전소 제약조건 (2)
for i in range(num_nodes_10M):
    for j in range(num_sub_10M):
        if i!=j:
            model_10M.addConstr(y[i, j] <= u[j], 'one_incomming_substation')
            
# string 제약 (터빈에서 하나만 나가도록)
for h in range(9, num_nodes_10M):
    model_10M.addConstr(gp.quicksum(y[h, j] for j in range(num_nodes) if j != h)==1)

# string 제약 (변전소에 들어오는 케이블 개수 제한)
for h in range(9):
    model_10M.addConstr(gp.quicksum(y[i, h] for i in range(9, num_nodes)) == num_strings * u[h])

# string 제약 (터빈의 들어오는 케이블이 최대 한개)
for h in range(9, num_nodes_10M):
    model_10M.addConstr(gp.quicksum(y[i, h] for j in range(num_nodes) if j != h) <= 1)

# string 제약 (최대 흐름 제한)
max_flow = math.ceil((num_turbines) / num_strings)
for i in range(num_nodes_10M):
     for j in range(num_nodes_10M):
          if i != j:
               model_10M.addConstr(f[i, j] <= max_flow)

# 외부 변전소만 선택
# 모든 변전소를 후보로 고려한다면 삭제
inside_sub = [0, 2, 3, 4]
for i in inside_sub:
    model_10M.addConstr(u[i] == 0)

# 콜백 함수 포함 최적화 호출
# 최적화 과정 중 Crossing이 발견 되었을 때, Lazy Constraint 추가 
model_10M.optimize(no_crossing_callback)

Set parameter TimeLimit to value 3600
Set parameter LazyConstraints to value 1
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i5-13600KF, instruction set [SSE2|AVX|AVX2]
Thread count: 14 physical cores, 20 logical processors, using up to 20 threads

Optimize a model with 7671 rows, 16816 columns and 39974 nonzeros
Model fingerprint: 0xef32a5bd
Variable types: 2401 continuous, 14415 integer (14415 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+01]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Presolve removed 4684 rows and 5163 columns
Presolve time: 0.06s
Presolved: 2987 rows, 11653 columns, 24743 nonzeros
Variable types: 1770 continuous, 9883 integer (9883 binary)

Root relaxation: objective 7.959558e+03, 3909 iterations, 0.05 seconds (0.15 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  O

In [9]:
print(f'Optimal solution found with total cost : {model_10M.objVal}')
print(f'Number of decision variables : {model_10M.NumVars}')
print(f'Number of constraints : {model_10M.NumConstrs}')

connections = []
for i in range(num_nodes_10M):
    for j in range(num_nodes_10M):
        if i != j and y[i, j].x > 0.5:
            for t in cable_types:
                if x[i, j, t].x > 0.5:
                    connections.append((i, j, t))

Optimal solution found with total cost : 8989.57636171469
Number of decision variables : 16816
Number of constraints : 7671


# 최적화 결과 시각화

In [ ]:
plt.figure(figsize=(15, 5))
plt.scatter(turbine_pos_10M[:, 0], turbine_pos_10M[:, 1], color='C0', label='10MW')
plt.scatter(substations_10M[:, 0], substations_10M[:, 1],  color='C1', label='10MW_OFFSUB')
plt.scatter(onshore_sub[0], onshore_sub[1], color='C2', label='10MW_ONSUB')

for s in range(len(turbine_pos_10M)):
    plt.text(turbine_pos_10M[s, 0], turbine_pos_10M[s, 1], str(s+1))

for itm, s in enumerate(substations_10M):
    plt.text(s[0], s[1], f'SUB_{itm+1}')
plt.text(onshore_sub[0], onshore_sub[1], 'ON_SUB')
cable_colors = {
    1: 'blue',
    2: 'green',
    3: 'orange',
    4: 'purple',
    5: 'red'
}

for (i, j, t) in connections:
    plt.plot([nodes_10M[i, 0], nodes_10M[j, 0]], [nodes_10M[i, 1], nodes_10M[j, 1]], color=cable_colors[t], label=f'cable_type_{t}')

plt.plot([nodes_10M[7, 0], onshore_sub[0]], [nodes_10M[7, 1], onshore_sub[1]], color = 'cyan', label='cable_type_6')

handles, labels = plt.gca().get_legend_handles_labels()
by_label = dict(zip(labels, handles))
plt.legend(by_label.values(), by_label.keys(), bbox_to_anchor=(1, 1))

plt.title('Optimized connections')
plt.show()    